In [2]:
import spotipy
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from dotenv import load_dotenv
import re
from tqdm import tqdm

load_dotenv('../secrets/.env', override=True)

%run useful_functions.ipynb
%run spotify_scraping.ipynb

spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

In [3]:
#shows = get_shows()
#show_broadcast_details = get_show_broadcast_details()
#broadcast_details = get_broadcast_details()

#master_df = (
#broadcast_playlists
#.merge(broadcast_details, on='Broadcast Key', how='left', suffixes=(" Broadcast Details",""))
#.merge(show_broadcast_details, on='Broadcast Key', how='left', suffixes=(""," Show Details"))
#.merge(shows, on='Show Key', how='left')
#.sort_values(['Service Date','Broadcast Position'], ascending = [False, True])
#)

In [4]:
broadcast_playlists = get_broadcast_playlists()
tracks = get_spotify_tracks()
albums = get_spotify_albums()
artists = get_spotify_artists()

tracks_to_run = broadcast_playlists[['Track Name', 'Artist Name']].sort_values(['Artist Name','Track Name'], ascending =  True).drop_duplicates(subset=['Artist Name','Track Name'], keep='first')
tracks_to_run = tracks_to_run.merge(tracks[['Original Search Artist','Original Search Track']], how='left', left_on=['Artist Name','Track Name'], right_on=['Original Search Artist','Original Search Track'], copy=False,indicator=True)
tracks_to_run = tracks_to_run[tracks_to_run['_merge']!="both"][['Artist Name','Track Name']]
tracks_to_run

,Artist Name,Track Name
0,,Faking Jazz Together
1,+HIRS+,Y'all Are Precious.
2,...From The Benthic Zone,Edwin
8,128kps,D4y n N1t3 (Phasmid's Unnecessary Mix)
15,24Hour Experience,Together
...,...,...
12673,Бони НЕМ,And I Love Her
12674,አለማየሁ እሸቴ,Alteleyeshegnem
12675,“Little” Louie Vega,All My Love
12682,上々颱風,Let It Be


In [6]:
#for i, (index, row) in enumerate(pbar := tqdm(tracks_to_run.iterrows())):
    #pbar.set_description(f"Iteration {i+1} of {len(tracks_to_run.index)} : {row['Artist Name']} : {row['Track Name']}")  
for i, (index, row) in enumerate(tracks_to_run[0:1].iterrows()):
    print("Iteration ",i+1," of ",len(tracks_to_run.index),":",row['Artist Name'],":", row['Track Name'])
    try:
        (temp_tracks,temp_albums,temp_artists) = get_track_info(row['Artist Name'],row['Track Name'])
    except:
        (temp_tracks,temp_albums,temp_artists) = (pd.DataFrame(),pd.DataFrame(),pd.DataFrame())
    tracks = pd.concat([tracks,temp_tracks])
    albums = pd.concat([albums,temp_albums])
    artists = pd.concat([artists,temp_artists])
    
del(i,
    index,
    row,
    temp_albums,
    temp_artists,
    temp_tracks,
    tracks_to_run)  

Iteration  1  of  1053 :  : Faking Jazz Together
Error with:   : Faking Jazz Together


In [7]:
tracks = tracks.drop_duplicates(subset=['Original Search Artist', 'Original Search Track'], keep='last')
albums = albums.drop_duplicates(subset='Album ID', keep='last')
artists = artists.drop_duplicates(subset='Artist ID', keep='last')

tracks.to_pickle("../data/spotify_tracks.pkl")
albums.to_pickle("../data/spotify_albums.pkl")
artists.to_pickle("../data/spotify_artists.pkl")

In [8]:
temp = tracks[['Track Name', 'Original Search Track', 'Original Search Artist']].applymap(lambda s: s.lower() if type(s) == str else s)
temp = temp[temp['Track Name'] != temp['Original Search Track']]
temp.to_excel('./temp.xlsx')
del(temp)
#CAN IMPROVE UPON MATCHING